<a href="https://colab.research.google.com/github/nibaskumar93n-debug/Morphoinformatics/blob/main/eggnog_sever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

# eggNOG-mapper in Google Colab

# Step 1: Install eggNOG-mapper with dependency fixes
# Install a newer compatible version of biopython first
!pip install biopython --upgrade

# Now install eggnog-mapper (it will use the newer biopython)
!pip install eggnog-mapper --no-deps
!pip install psutil xlsxwriter

# Step 2: Manually download eggNOG 6.0 database from the correct URLs
import os
os.makedirs('/content/eggnog_data', exist_ok=True)

print("Downloading eggNOG 6.0 database files manually...")
print("This may take 20-30 minutes depending on your connection.\n")

# Download the main database files from eggnog6 server
!cd /content/eggnog_data && wget -c http://eggnog6.embl.de/download/emapperdb-5.0.2/eggnog.db.gz && gunzip -f eggnog.db.gz
print("✓ Downloaded eggnog.db\n")

!cd /content/eggnog_data && wget -c http://eggnog6.embl.de/download/emapperdb-5.0.2/eggnog.taxa.tar.gz && tar -xzf eggnog.taxa.tar.gz && rm eggnog.taxa.tar.gz
print("✓ Downloaded eggnog.taxa.db\n")

!cd /content/eggnog_data && wget -c http://eggnog6.embl.de/download/emapperdb-5.0.2/eggnog_proteins.dmnd.gz && gunzip -f eggnog_proteins.dmnd.gz
print("✓ Downloaded eggnog_proteins.dmnd\n")

print("Database download complete!")

This may take 20-30 minutes depending on your connection.

--2026-01-15 06:57:01--  http://eggnog6.embl.de/download/emapperdb-5.0.2/eggnog.db.gz
Resolving eggnog6.embl.de (eggnog6.embl.de)... 194.94.44.170
Connecting to eggnog6.embl.de (eggnog6.embl.de)|194.94.44.170|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6776977123 (6.3G) [application/octet-stream]
Saving to: ‘eggnog.db.gz’

eggnog.db.gz         16%[==>                 ]   1.01G  14.9MB/s    in 71s     

2026-01-15 06:58:12 (14.7 MB/s) - Connection closed at byte 1086896014. Retrying.

--2026-01-15 06:58:13--  (try: 2)  http://eggnog6.embl.de/download/emapperdb-5.0.2/eggnog.db.gz
Connecting to eggnog6.embl.de (eggnog6.embl.de)|194.94.44.170|:80... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 6776977123 (6.3G), 5690081109 (5.3G) remaining [application/octet-stream]
Saving to: ‘eggnog.db.gz’

eggnog.db.gz         32%[+++==>              ]   2.02G  14.5MB/s    in 71s    

In [4]:
# Step 3: Upload your sequence file
from google.colab import files
print("Upload your FASTA file:")
uploaded = files.upload()

Upload your FASTA file:


Saving Roseburia_UP000049828_predicted_essential_revised_threshold.fasta to Roseburia_UP000049828_predicted_essential_revised_threshold.fasta


In [6]:
# Get the filename
input_file = list(uploaded.keys())[0]
print(f"Uploaded file: {input_file}")

Uploaded file: Roseburia_UP000049828_predicted_essential_revised_threshold.fasta


In [7]:
# Step 4: Run eggNOG-mapper
output_name = "eggnog_results"

!emapper.py -i {input_file} \
  -o {output_name} \
  --data_dir /content/eggnog_data \
  -m diamond \
  --cpu 2

#  emapper-2.1.13
# emapper.py  -i Roseburia_UP000049828_predicted_essential_revised_threshold.fasta -o eggnog_results --data_dir /content/eggnog_data -m diamond --cpu 2
  /usr/local/lib/python3.12/dist-packages/eggnogmapper/bin/diamond blastp -d '/content/eggnog_data/eggnog_proteins.dmnd' -q '/content/Roseburia_UP000049828_predicted_essential_revised_threshold.fasta' --threads 2 -o '/content/eggnog_results.emapper.hits' --tmpdir '/content/emappertmp_dmdn_yu4btwdw' --sensitive --iterate -e 0.001 --top 3  --outfmt 6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovhsp scovhsp
Functional annotation of hits...
461 40.849878787994385 11.29 q/s (% mem usage: 8.80, % mem avail: 91.18)
Done
Result files:
   /content/eggnog_results.emapper.hits
   /content/eggnog_results.emapper.seed_orthologs
   /content/eggnog_results.emapper.annotations

CITATION:
If you use this software, please cite:

[1] eggNOG-mapper v2: functional annotation, orthology assignment

In [8]:
import pandas as pd
from google.colab import files
import os

# Specify your annotation file path (change this to your actual filename)
annotation_file = "eggnog_results.emapper.annotations"  # Change this to your file name

print(f"Processing: {annotation_file}\n")

# Read the annotation file (skip comment lines starting with #)
try:
    df = pd.read_csv(annotation_file, sep='\t', comment='#', header=None)

    # Define column names based on eggNOG-mapper output format
    column_names = [
        'query', 'seed_ortholog', 'evalue', 'score', 'eggNOG_OGs',
        'max_annot_lvl', 'COG_category', 'Description', 'Preferred_name',
        'GOs', 'EC', 'KEGG_ko', 'KEGG_Pathway', 'KEGG_Module',
        'KEGG_Reaction', 'KEGG_rclass', 'BRITE', 'KEGG_TC', 'CAZy',
        'BiGG_Reaction', 'PFAMs'
    ]

    # Assign column names (use only as many as exist in the file)
    df.columns = column_names[:len(df.columns)]

    print(f"✓ Successfully parsed {len(df)} sequences\n")

    # Create output filename
    base_name = annotation_file.replace('.emapper.annotations', '')
    excel_file = f"{base_name}_annotations.xlsx"
    csv_file = f"{base_name}_annotations.csv"

    # Save as Excel
    print("Creating Excel file...")
    df.to_excel(excel_file, index=False, sheet_name='Annotations')
    print(f"✓ Created: {excel_file}")

    # Save as CSV
    print("Creating CSV file...")
    df.to_csv(csv_file, index=False)
    print(f"✓ Created: {csv_file}\n")

    # Download files
    print("Downloading files...")
    files.download(excel_file)
    files.download(csv_file)

    print("\n✓ Done!")

except Exception as e:
    print(f"ERROR: {e}")

Processing: eggnog_results.emapper.annotations

✓ Successfully parsed 461 sequences

Creating Excel file...
✓ Created: eggnog_results_annotations.xlsx
Creating CSV file...
✓ Created: eggnog_results_annotations.csv



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✓ Done!
